This is how we play around with less API requests to the forked/fake networks
A tinkering environment...


In [1]:
from moccasin import setup_notebook

setup_notebook()

In [6]:
from moccasin.config import get_active_network

active_network = get_active_network()
print(active_network.name)

eth-forked


In [7]:
# some network checks
# if we are running this on a test/forked network, let's give us some fake money

from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network, Network
import boa

STARTING_ETH_BALANCE = int(1000e18)
STARTING_USDC_BALANCE = int(100e6)
STARTING_WETH_BALANCE = int(1e18)


def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)


def _add_token_balance(usdc: ABIContract, weth: ABIContract, active_network: Network):
    our_address = boa.env.eoa
    # we can use usdc contract as a proxy and pretend to be the owner
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    # because we now have the abi, we can use it to mint WETH
    weth.deposit(value=STARTING_WETH_BALANCE)


# these ABIContracts classes are to help us call contracts on blockchain when we don't know exactly the source code
def setup_script() -> Tuple[ABIContract, ABIContract, ABIContract, ABIContract]:
    print("Setting up script")

    # 1. give ourselves some ETH
    # 2. give ourselves some USDC and WETH

    active_network = get_active_network()

    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")

    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth, active_network)


def moccasin_main():
    setup_script()


moccasin_main()

Setting up script


In [8]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [10]:
usdc.balanceOf(boa.env.eoa)
weth.balanceOf(boa.env.eoa)

2000000000000000000

In [ ]:
from moccasin.config import get_config

# this bit because we added new abis , need to reload and refresh the config
config = get_config()
config.reload()
active_network = config.get_active_network()
aavev3_pool_address_provider = active_network.manifest_named(
    "aavev3_pool_address_provider"
)
# aave v3 address provider provides the pool address
# it's some master address that keeps the changing pool address
pool_address = aavev3_pool_address_provider.getPool()
# print(pool_address) # this is the address we send money to
# now we need to get abis from this

0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [18]:
from moccasin.config import get_config

# this bit because we added new abis , need to reload and refresh the config
config = get_config()
config.reload()
active_network = config.get_active_network()
pool_contract = active_network.manifest_named("pool", address=pool_address)
pool_contract

<pool interface at 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2>

In [19]:
REFERRAL_CODE = 0


def deposit(pool_contract, token, amount):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {amount} {token.symbol} to {pool_contract.address}")
    pool_contract.supply(
        token.address, amount, boa.env.eoa, REFERRAL_CODE
    )  # see the supply documentation for more details


usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)

    # check aave doc for more details on the getUSerAccountData function
    # collateral will be the amount we deposit even if we are not borrowing
    (
        totalCollateralBase,
        totalDebtBase,
        availableBorrowsBase,
        currentLiquidationThreshold,
        ltv,
        healthFactor,
    ) = pool_contract.getUserAccountData(boa.env.eoa)
    print(f"""User account data:
        totalCollateralBase: {totalCollateralBase}
        totalDebtBase: {totalDebtBase}
        availableBorrowsBase: {availableBorrowsBase}
        currentLiquidationThreshold: {currentLiquidationThreshold}
        ltv: {ltv}
        healthFactor: {healthFactor}
          """)

Depositing 300000000 ABI usdc.symbol() -> ['string'] to 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
Depositing 2000000000000000000 ABI weth.symbol() -> ['string'] to 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
User account data:
        totalCollateralBase: 719644781340
        totalDebtBase: 0
        availableBorrowsBase: 577658865982
        currentLiquidationThreshold: 8279
        ltv: 8027
        healthFactor: 115792089237316195423570985008687907853269984665640564039457584007913129639935
          


In [21]:
# refresh
config.reload()
active_network = config.get_active_network()
aave_protocol_data_provider = active_network.manifest_named(
    "aave_protocol_data_provider"
)
a_tokens = aave_protocol_data_provider.getAllATokens()
print(a_tokens)

[('aEthWETH', Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), ('aEthwstETH', Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), ('aEthWBTC', Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), ('aEthUSDC', Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), ('aEthDAI', Address('0x018008bfb33d285247A21d44E50697654f754e63')), ('aEthLINK', Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), ('aEthAAVE', Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), ('aEthcbETH', Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), ('aEthUSDT', Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), ('aEthrETH', Address('0xCc9EE9483f662091a1de4795249E24aC0aC2630f')), ('aEthLUSD', Address('0x3Fe6a295459FAe07DF8A0ceCC36F37160FE86AA9')), ('aEthCRV', Address('0x7B95Ec873268a6BFC6427e7a28e396Db9D0ebc65')), ('aEthMKR', Address('0x8A458A9dc9048e005d22849F470891b840296619')), ('aEthSNX', Address('0xC7B4c17861357B8ABB91F25581E7263E08DCB59c')), ('aEthBAL', Address('0x2516E7B3F76

In [24]:
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_network.manifest_named("weth", address=a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_network.manifest_named("usdc", address=a_token[1])

print(a_usdc)
print(a_weth)

<usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>
<weth interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>


In [27]:
# Get how much they are worth to figure out what our portfolio makeup is
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)  # 6 decimals
a_weth_balance = a_weth.balanceOf(boa.env.eoa)  # 18 decimals

a_usdc_balance_normalized = a_usdc_balance / 1e6
a_weth_balance_normalized = a_weth_balance / 1e18
print(f"Normalized USDC balance: {a_usdc_balance_normalized}")
print(f"Normalized WETH balance: {a_weth_balance_normalized}")

Normalized USDC balance: 300.0
Normalized WETH balance: 2.0


In [31]:
config.reload()


def get_price(feed_name: str) -> float:
    active_network = get_active_network()
    price_feed = active_network.manifest_named(feed_name)
    price = price_feed.latestAnswer()
    decimals = price_feed.decimals()
    return price / (10**decimals)


usdc_price = get_price("usdc_usd")
weth_price = get_price("eth_usd")
print(f"USDC price: {usdc_price}")
print(f"WETH price: {weth_price}")

USDC price: 0.99995232
WETH price: 3448.2310587


In [33]:
usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price
total_value = usdc_value + weth_value

target_usdc_ratio = 0.3
target_weth_ratio = 0.7


usdc_allocation_ratio = usdc_value / total_value
weth_allocation_ratio = weth_value / total_value

BUFFER = 0.1

needs_rebalancing = (abs(usdc_allocation_ratio - target_usdc_ratio) > BUFFER) or (
    abs(weth_allocation_ratio - target_weth_ratio) > BUFFER
)
print(needs_rebalancing)
print(f"USDC allocation ratio: {usdc_allocation_ratio*100}%")
print(f"WETH allocation ratio: {weth_allocation_ratio*100}%")

True
USDC allocation ratio: 4.168524580160475%
WETH allocation ratio: 95.83147541983953%


In [35]:
# we only need to sell eth and buy usdc, and we only need to sell prat of it not all
# but to make it easier we withdraw all weth
a_weth.approve(pool_contract.address, a_weth.balanceOf(boa.env.eoa))
# we need to give aweth to recevie eth, so we need to approve the pool contract to spend our aWETH
pool_contract.withdraw(weth.address, a_weth.balanceOf(boa.env.eoa), boa.env.eoa)


def print_token_balances():
    print(f"USDC balance: {usdc.balanceOf(boa.env.eoa)}")
    print(f"WETH balance: {weth.balanceOf(boa.env.eoa)}")
    print(f"aUSDC balance: {a_usdc.balanceOf(boa.env.eoa)}")
    print(f"aWETH balance: {a_weth.balanceOf(boa.env.eoa)}")


print_token_balances()

USDC balance: 0
WETH balance: 2000000000000000000
aUSDC balance: 300000000
aWETH balance: 0


In [44]:
# this chunk calculates how much to sell/buy
usdc_data = {
    "balance": a_usdc_balance_normalized,
    "price": usdc_price,
    "contract": a_usdc,
}
weth_data = {
    "balance": a_weth_balance_normalized,
    "price": weth_price,
    "contract": a_weth,
}
target_allocations = {"usdc": 0.3, "weth": 0.7}


def calculate_rebalancing_trades(
    usdc_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    weth_data: dict,  # {"balance": float, "price": float, "contract": Contract}
    target_allocations: dict[str, float],  # {"usdc": 0.3, "weth": 0.7}
) -> dict[str, dict]:
    """
    Calculate the trades needed to rebalance a portfolio of USDC and WETH.

    Args:
        usdc_data: Dict containing USDC balance, price and contract
        weth_data: Dict containing WETH balance, price and contract
        target_allocations: Dict of token symbol to target allocation (must sum to 1)

    Returns:
        Dict of token symbol to dict containing contract and trade amount:
            {"usdc": {"contract": Contract, "trade": int},
             "weth": {"contract": Contract, "trade": int}}
    """
    # Calculate current values
    usdc_value = usdc_data["balance"] * usdc_data["price"]
    weth_value = weth_data["balance"] * weth_data["price"]
    total_value = usdc_value + weth_value

    # Calculate target values
    target_usdc_value = total_value * target_allocations["usdc"]
    target_weth_value = total_value * target_allocations["weth"]

    # Calculate trades needed in USD
    usdc_trade_usd = target_usdc_value - usdc_value
    weth_trade_usd = target_weth_value - weth_value

    # Convert to token amounts
    return {
        "usdc": {
            "contract": usdc_data["contract"],
            "trade": usdc_trade_usd / usdc_data["price"],
        },
        "weth": {
            "contract": weth_data["contract"],
            "trade": weth_trade_usd / weth_data["price"],
        },
    }


trades = calculate_rebalancing_trades(usdc_data, weth_data, target_allocations)
print(trades)

weth_to_sell = trades["weth"]["trade"]
print(weth_to_sell)

{'usdc': {'contract': <usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>, 'trade': 1859.0372869178402}, 'weth': {'contract': <weth interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>, 'trade': -0.539102112467148}}
-0.539102112467148


In [49]:
# interact with Uniswap, we will here use V3
# checkout implement a swap part of the docs

# struct ExactInputSingleParams {
#     address tokenIn;
#     address tokenOut;
#     uint24 fee;
#     address recipient;
#     uint256 deadline;
#     uint256 amountIn;
#     uint256 amountOutMinimum;
#     uint160 sqrtPriceLimitX96;
# }

config.reload()
active_network = config.get_active_network()
uniswap_swap_router = active_network.manifest_named("uniswap_swap_router")

amount_weth = abs(int(weth_to_sell * (10**18)))
print(amount_weth)

weth.approve(uniswap_swap_router.address, amount_weth)

# this helps prevent against MEV
min_out = int((trades["usdc"]["trade"] * (10**6)) * 0.9)

uniswap_swap_router.exactInputSingle(
    (
        weth.address,  # "tokenIn"
        usdc.address,  # "tokenOut"
        3000,  # "fee": 0.3% fee hardcoded
        boa.env.eoa,  # "recipient"
        amount_weth,  # "amountIn"
        min_out,  # "amountOutMinimum"
        0,  # "sqrtPriceLimitX96"
    )
)

print_token_balances()


539102112467148032
USDC balance: 1856558156
WETH balance: 1460897887532851968
aUSDC balance: 300000000
aWETH balance: 0


In [50]:
# redeposit back to aave

amount = usdc.balanceOf(boa.env.eoa)
deposit(pool_contract, usdc, amount)

print_token_balances()

Depositing 1856558156 ABI usdc.symbol() -> ['string'] to 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
USDC balance: 0
WETH balance: 1460897887532851968
aUSDC balance: 2156558156
aWETH balance: 0


In [51]:
deposit(pool_contract, weth, weth.balanceOf(boa.env.eoa))

Depositing 1460897887532851968 ABI weth.symbol() -> ['string'] to 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
